In [28]:

pip install spellchecker

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: C:\Users\Usuario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [29]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
file = 'todas_receitas_porcoeslimpas.csv'
df = pd.read_csv(file, index_col=0)

<h2>Limpeza</h2>

In [31]:
def split_ingredients(ingredients_text):
    ingredients_text = re.sub(r"[\{\}]", "", ingredients_text).strip()
    ingredients_list = [item.strip() for item in ingredients_text.split(',')]
    return ingredients_list

df['Ingredientes'] = df['Ingredientes'].apply(split_ingredients)

In [32]:
df.head(10)

,Porções,Ingredientes,Instruções
Receita,,,
Massa de Pizza,16,"['1/2 kg de farinha de trigo', '1 copo de água...","{'1. Dissolva o fermento na água morna.', '2. ..."
Receita de Bolo de Cenoura,24,"['1 pitada de sal', '3 unidades de cenoura méd...","{'1. Bata no liquidificador os ovos, as cenour..."
Bolo de Chocolate Molhadinho,16,"['1 xícara (chá) de leite', '3 unidades de ovo...",{'1. Bata bem todos os ingredientes no liquidi...
Como Fazer Bolinho de Chuva,14,"['1 colher (sopa) de fermento químico em pó', ...",{'1. Bata o açúcar com os ovos e junte os dema...
Bolo de Fubá,20,"['2 copos de fubá', '1 copo de farinha de trig...","{'1. Bater tudo no liquidificador.', '2. Por ú..."
Bolo de Milho,12,"['1 lata de milho em conserva', '3 unidades de...",{'1. Bata todos os ingredientes no liquidifica...
Fricassê de Frango,8,['600 gr de peito de frango cozido e desfiado'...,{'1. Bata no liquidificador o milho e o creme ...
Bolo de Aniversário,15,"['4 unidades de ovo', '2 xícaras (chá) de açúc...","{'1. Na batedeira bata as claras em neve, junt..."
Bolo de Banana,12,"['3 unidades de banana nanica', '3 unidades de...","{'1. Em um liquidificador coloque os ovos, a b..."


Removendo números

In [33]:
def remove_numbers(ingredients):
  return [re.sub(r'\d+\s*', '', item).strip() for item in ingredients]

df['Ingredientes'] = df['Ingredientes'].apply(remove_numbers)

In [34]:
df.head(10)

,Porções,Ingredientes,Instruções
Receita,,,
Massa de Pizza,16,"['/kg de farinha de trigo', 'copo de água', 'g...","{'1. Dissolva o fermento na água morna.', '2. ..."
Receita de Bolo de Cenoura,24,"['pitada de sal', 'unidades de cenoura médias ...","{'1. Bata no liquidificador os ovos, as cenour..."
Bolo de Chocolate Molhadinho,16,"['xícara (chá) de leite', 'unidades de ovo', '...",{'1. Bata bem todos os ingredientes no liquidi...
Como Fazer Bolinho de Chuva,14,"['colher (sopa) de fermento químico em pó', 'x...",{'1. Bata o açúcar com os ovos e junte os dema...
Bolo de Fubá,20,"['copos de fubá', 'copo de farinha de trigo', ...","{'1. Bater tudo no liquidificador.', '2. Por ú..."
Bolo de Milho,12,"['lata de milho em conserva', 'unidades de ovo...",{'1. Bata todos os ingredientes no liquidifica...
Fricassê de Frango,8,"['gr de peito de frango cozido e desfiado', 'c...",{'1. Bata no liquidificador o milho e o creme ...
Bolo de Aniversário,15,"['unidades de ovo', 'xícaras (chá) de açúcar',...","{'1. Na batedeira bata as claras em neve, junt..."
Bolo de Banana,12,"['unidades de banana nanica', 'unidades de ovo...","{'1. Em um liquidificador coloque os ovos, a b..."


Todas as letras em minúsculas

In [35]:
def convert_lowercase(ingredients):
    return [item.lower().strip() for item in ingredients]

df['Ingredientes'] = df['Ingredientes'].apply(convert_lowercase)

Remover aspas

In [36]:
def remove_quotes_from_words(ingredient_list):
    cleaned_ingredients = []
    for ingredients in ingredient_list:
        cleaned_ingredients.append([ingredient.strip("'") for ingredient in ingredients])
    return cleaned_ingredients

df['Ingredientes'] = remove_quotes_from_words(df['Ingredientes'])

Remover stopwords

In [37]:
def clean_stopwords(ingredient_list):
    stop_words = set(stopwords.words('portuguese'))
    # print(f"Stopwords carregadas: {stop_words}")
    cleaned_ingredients = []
    for ingredient in ingredient_list:
        ingredient = ingredient.strip()
        words = ingredient.split()
        filtered_words = [word for word in words if word.lower() not in stop_words]
        cleaned_ingredient = ' '.join(filtered_words)
        cleaned_ingredients.append(cleaned_ingredient)
    return cleaned_ingredients

df['Ingredientes'] = df['Ingredientes'].apply(clean_stopwords)

In [38]:
df.head(10)

,Porções,Ingredientes,Instruções
Receita,,,
Massa de Pizza,16,"[/kg farinha trigo, copo água, gr fermento bio...","{'1. Dissolva o fermento na água morna.', '2. ..."
Receita de Bolo de Cenoura,24,"[pitada sal, unidades cenoura médias cruas, xí...","{'1. Bata no liquidificador os ovos, as cenour..."
Bolo de Chocolate Molhadinho,16,"[xícara (chá) leite, unidades ovo, colheres (s...",{'1. Bata bem todos os ingredientes no liquidi...
Como Fazer Bolinho de Chuva,14,"[colher (sopa) fermento químico pó, xícara (ch...",{'1. Bata o açúcar com os ovos e junte os dema...
Bolo de Fubá,20,"[copos fubá, copo farinha trigo, copo óleo mil...","{'1. Bater tudo no liquidificador.', '2. Por ú..."
Bolo de Milho,12,"[lata milho conserva, unidades ovo, /xícaras (...",{'1. Bata todos os ingredientes no liquidifica...
Fricassê de Frango,8,"[gr peito frango cozido desfiado, colheres (so...",{'1. Bata no liquidificador o milho e o creme ...
Bolo de Aniversário,15,"[unidades ovo, xícaras (chá) açúcar, /copos fa...","{'1. Na batedeira bata as claras em neve, junt..."
Bolo de Banana,12,"[unidades banana nanica, unidades ovo, xícaras...","{'1. Em um liquidificador coloque os ovos, a b..."


Remover medidas e adjetivos

In [39]:
def clean_measurements(ingredients):
    cleaned_ingredients = []
    for item in ingredients:
        item = re.sub(r"\d+|\b(acrescente|american|aproximadamente|batida|batidas|batido|bem|bola|bolas|caixa|caixas|caixinha|caixinhas|chá|cheia|cheias|colher|colheres|copo|copos|cozida|cozidas|cozido|cozidos|crua|cruas|decorar|dente|dentes|derretida|derretidas|derretido|derretidos|desfiado|desfiados|eisenbahn|embebido|envelope|envelopes|fatia|fatias|forte|gosto|grosso|grossa|\bgr\b|grs|grama|gramas|kg|kgs|lactose|lata|latas|litro|litros|maço|maços|magro|média|médias|ml|mls|mililitro|mililitros|morno|orgânico|orgânicos|pacote|pacotes|peneirada|peneiradas|picado|picados|picada|picadas|picadinha|picadinhas|pincelar|pitada|pitadas|polvilhar|quilo|quilos|raspa|raspas|refogar|rodela|rodelas|sobremesa|sopa|sopas|tablete|tabletes|unidade|unidades|xícara|xícaras|zero|esfarelado|especial|fatiada|fresco|grande|grudar|ipa|liga|lipidios|malte|meio|morna|mãos|padrão|pode|pote|preferência|puro|ralada|ralado|recheio|suficiente|tostado|trituradas|triturado|untar|vidro|branca|brancas|branco|brancos|da|de|do|dar|maduro|maduros|madura|maduras|picadinho|picadinhos|picadinha|picadinhas|casca|s|o|gema|gemas|jasmine|mix|gelada|geladas|gelado|gelados)\b", "", item)
        item = item.replace('-', ' ')
        tokens = [word for word in item.split()]
        cleaned_ingredients.append(" ".join(tokens))

    return cleaned_ingredients

df['Ingredientes'] = df['Ingredientes'].apply(clean_measurements)

In [40]:
df.head(10)

,Porções,Ingredientes,Instruções
Receita,,,
Massa de Pizza,16,"[/ farinha trigo, água, fermento biológico, / ...","{'1. Dissolva o fermento na água morna.', '2. ..."
Receita de Bolo de Cenoura,24,"[sal, cenoura, () óleo soja, () fermento quími...","{'1. Bata no liquidificador os ovos, as cenour..."
Bolo de Chocolate Molhadinho,16,"[() leite, ovo, () margarina, () açúcar, () ch...",{'1. Bata bem todos os ingredientes no liquidi...
Como Fazer Bolinho de Chuva,14,"[() fermento químico pó, () farinha trigo, ovo...",{'1. Bata o açúcar com os ovos e junte os dema...
Bolo de Fubá,20,"[fubá, farinha trigo, óleo milho, leite, / açú...","{'1. Bater tudo no liquidificador.', '2. Por ú..."
Bolo de Milho,12,"[milho conserva, ovo, / () fubá, / () açúcar, ...",{'1. Bata todos os ingredientes no liquidifica...
Fricassê de Frango,8,"[peito frango, () óleo soja, sal, pimenta rein...",{'1. Bata no liquidificador o milho e o creme ...
Bolo de Aniversário,15,"[ovo, () açúcar, / farinha trigo, leite, () ma...","{'1. Na batedeira bata as claras em neve, junt..."
Bolo de Banana,12,"[banana nanica, ovo, () farinha trigo, / () am...","{'1. Em um liquidificador coloque os ovos, a b..."


Remover caracteres não-alfabéticos

In [41]:
def clean_non_alpha(ingredients):
    cleaned_ingredients = []
    for item in ingredients:
        item = re.sub(r'[^a-zA-Zá-úÁ-Ú\s]', '', item)
        item = item.replace('-', ' ')
        tokens = [word for word in item.split()]
        cleaned_ingredients.append(" ".join(tokens))

    return cleaned_ingredients
df['Ingredientes'] = df['Ingredientes'].apply(clean_non_alpha)

In [42]:
df.head(10)

,Porções,Ingredientes,Instruções
Receita,,,
Massa de Pizza,16,"[farinha trigo, água, fermento biológico, óleo...","{'1. Dissolva o fermento na água morna.', '2. ..."
Receita de Bolo de Cenoura,24,"[sal, cenoura, óleo soja, fermento químico pó,...","{'1. Bata no liquidificador os ovos, as cenour..."
Bolo de Chocolate Molhadinho,16,"[leite, ovo, margarina, açúcar, chocolate pó, ...",{'1. Bata bem todos os ingredientes no liquidi...
Como Fazer Bolinho de Chuva,14,"[fermento químico pó, farinha trigo, ovo, leit...",{'1. Bata o açúcar com os ovos e junte os dema...
Bolo de Fubá,20,"[fubá, farinha trigo, óleo milho, leite, açúca...","{'1. Bater tudo no liquidificador.', '2. Por ú..."
Bolo de Milho,12,"[milho conserva, ovo, fubá, açúcar, leite, óle...",{'1. Bata todos os ingredientes no liquidifica...
Fricassê de Frango,8,"[peito frango, óleo soja, sal, pimenta reino, ...",{'1. Bata no liquidificador o milho e o creme ...
Bolo de Aniversário,15,"[ovo, açúcar, farinha trigo, leite, manteiga, ...","{'1. Na batedeira bata as claras em neve, junt..."
Bolo de Banana,12,"[banana nanica, ovo, farinha trigo, amido milh...","{'1. Em um liquidificador coloque os ovos, a b..."


<h2>Normalização</h2>

Correção de ortografia - essa biblioteca só fez cagada

In [43]:
# def correct_spelling(ingredients):
#     corrected_words = []
#     corrections = {}  # Dicionário para armazenar correções
#     for ingredient in ingredients:
#         words = ingredient.split()
#         corrected_ingredient = []
#         for word in words:
#             corrected_word = spell.correction(word)
#             if corrected_word != word:
#                 corrections[word] = corrected_word
#             corrected_ingredient.append(corrected_word if corrected_word else word)
#         corrected_words.append(" ".join(corrected_ingredient))
#     print("Palavras corrigidas:", corrections)
#     return corrected_words

# df['Ingredientes'] = df['Ingredientes'].apply(correct_spelling)

Colocar todas as palavras no singular

In [44]:
lemmatizer = WordNetLemmatizer()

def singularize_ingredients(ingredients, recipe_name):
    modified_words = {}
    singularized_ingredients = []

    for ingredient in ingredients:
        words = ingredient.split()
        singularized_ingredient = []

        for word in words:
            singular_word = lemmatizer.lemmatize(word, pos='n')
            if singular_word != word:
                modified_words[word] = singular_word
            singularized_ingredient.append(singular_word)

        singularized_ingredients.append(" ".join(singularized_ingredient))

    if modified_words:
        print(f"Receita: {recipe_name}")
        print("Palavras modificadas:", modified_words)

    return singularized_ingredients

df['Ingredientes'] = df.apply(lambda row: singularize_ingredients(row['Ingredientes'], row['Receita']), axis=1)

# df['Ingredientes'] = df['Ingredientes'].apply(singularize_ingredients)

KeyError: 'Receita'

In [ ]:
df.head(10)

,Receita,Ingredientes
0,Massa de Pizza,"[farinha trigo, água, fermento biológico, óleo..."
1,Receita de Bolo de Cenoura,"[sal, cenoura, óleo soja, fermento químico pó,..."
2,Bolo de Chocolate Molhadinho,"[leite, ovo, margarina, açúcar, chocolate pó, ..."
3,Como Fazer Bolinho de Chuva,"[fermento químico pó, farinha trigo, ovo, leit..."
4,Bolo de Fubá,"[fubá, farinha trigo, óleo milho, leite, açúca..."
5,Bolo de Milho,"[milho conserva, ovo, fubá, açúcar, leite, óle..."
6,Fricassê de Frango,"[peito frango, óleo soja, sal, pimenta reino, ..."
7,Bolo de Aniversário,"[ovo, açúcar, farinha trigo, leite, manteiga, ..."
8,Bolo de Banana,"[banana nanica, ovo, farinha trigo, amido milh..."
9,Panquecas,"[farinha trigo, leite, amido milho, ovo, sal, ..."


In [ ]:
# df.to_csv('ingredientes_parciais.csv')

<h2>Todos os ingredientes</h2>

In [ ]:
# Está imprimindo cada palavra separadamente
# all_ingredients = set()
# for ingredients_list in df['Ingredientes']:
#     for ingredient in ingredients_list:
#         all_ingredients.update(ingredient.split())

# print("Todos os ingredientes presentes no DataFrame:")
# print(sorted(all_ingredients))

In [ ]:
all_ingredients = set()
for ingredients_list in df['Ingredientes']:
    all_ingredients.update(ingredients_list)

ingredients_df = pd.DataFrame(all_ingredients, columns=['Ingrediente'])
ingredients_df.to_csv('todos_ingredientes.csv')